In [ ]:
import numpy as np
import tables
import matplotlib.pyplot as plt
import pickle
import os
from icecube import icetray


In [ ]:
def calc_effa(mc, coszen_min, coszen_max, bins_log10E, n_events):
        """
        Calculate the effective area of the dataset

        mc: numpy.ndarray
                Simulation data
        coszen_min: float, radians
                Starting zenith of your area of sky
        coszen_max: float, radians
                Ending zenith of your area of sky
        bins_log10E: list
                The energy bins in log space
        """
        
        # Prepare the pieces that go into the effective area
        solid_ang = 2.0 * np.pi * (coszen_max - coszen_min)
        coszen = np.cos(np.asarray(mc.root.I3MCWeightDict.cols.PrimaryNeutrinoZenith[:]))
        true_E = np.asarray(mc.root.I3MCWeightDict.cols.PrimaryNeutrinoEnergy[:])
        #ow = mc['OneWeight']         # Simulation OneWeight/Nevents (GeV cm^2 sr)
        ow = mc.root.I3MCWeightDict.cols.OneWeight[:]
        print("Coszen select:",coszen_min,coszen_max)
        # Create a mask that selects the dec band
        alert_filter_raw = np.asarray(mc.root.pass_gfu.cols.value[:])
        alert_filter = []
        for item in alert_filter_raw:
            alert_filter.append(item>0.5)
        mask = (coszen_min <= coszen) & (coszen < coszen_max) & alert_filter
        print("Events:",len(ow[mask]), len(ow))
        
        # Weights, bins, and histogram
        bin_width_log10E = np.diff(bins_log10E)
        weights_Aeff = (ow[mask] * 1e-4) / (solid_ang*n_events)
        h = np.histogram(true_E[mask], bins=bins_log10E, weights=weights_Aeff)[0]/bin_width_log10E
        return h


In [ ]:
#hard coded for mc sample:
#n_files = 200
n_files = 9995
#n_files = 200
#n_events = 200000
#n_events = 2.5E2
#max_E_log = 8.0
#min_E_log = 2.0
#max_zenith = np.pi
#min_zenith = 0.0

bins_log10E = np.logspace(2, 9, 25)
color_list = ['b', 'y', 'g', 'r', 'm']

# Prepare the figure and subplot
plt.figure(figsize=(10,8), dpi=300)
ax1 = plt.subplot (1, 1, 1)

# Load the MC for each dataset
#exp, mc, livetime = Datasets[dataset].season(season)

#mc = tables.open_file('./200files_test.hdf5')
mc = tables.open_file('./9995file_20878.hdf5')

#I3MCWeightDict knows things...
n_events = mc.root.I3MCWeightDict.cols.NEvents[0]
max_E_log = mc.root.I3MCWeightDict.cols.MaxEnergyLog[0]
min_E_log = mc.root.I3MCWeightDict.cols.MinEnergyLog[0]
max_zenith = mc.root.I3MCWeightDict.cols.MaxZenith[0]
min_zenith = mc.root.I3MCWeightDict.cols.MinZenith[0]
print(n_events, max_E_log, min_E_log, max_zenith, min_zenith)
# Bins
#bins_coszen = np.linspace(-1.0,1.0,6)
#bins_coszen = [180, 120, 95, 90, 60, 0]
bins_coszen = [180, 0]


# Get the histogram and plot the effective area for each zenith band
for (i, (coszen_min, coszen_max)) in enumerate (zip (bins_coszen[:-1], bins_coszen[1:])):
        h1 = calc_effa(mc, np.cos(np.radians(coszen_min)), np.cos(np.radians(coszen_max)), bins_log10E, n_events* n_files)
        ax1.plot (bins_log10E, np.r_[h1, h1[-1]], drawstyle='steps-post', color=color_list[i], linewidth=2,
                  label=r'${:.2f} \leq \delta  < {:.2f}$'.format (((coszen_max-90)), ((coszen_min-90.0))))

# Plot the overall stuff
ax1.set_ylim(1e-1,1e4)  # was 1e-3
ax1.set_xlim(1e2, 5e8)

plt.grid(True)
ax1.set_yscale('log')
ax1.set_xscale('log')
plt.suptitle('Effective Area GFU Online', size=18)
ax1.set_ylabel(r'Effective Area (m$^{2}$)', fontsize=18)
ax1.set_xlabel('Neutrino Energy (GeV)', fontsize=18)

ax1.legend(loc='upper left', ncol=1, fancybox=True, shadow=True)

# Set grid lines
for ymaj in ax1.yaxis.get_majorticklocs():
        ax1.axhline (y=ymaj,ls=':', color='gray', alpha =0.75, linewidth=0.5)
for xmaj in ax1.xaxis.get_majorticklocs():
        ax1.axvline (x=xmaj,ls=':', color='gray', alpha =0.75, linewidth=0.5)
        
plt.savefig('gfu_effa2.png')